# Inter-Uni Datathon: Team Habaybi

### Data Cleaning

In [1]:
# import required packages

import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')



In [2]:
train.describe()

,TransactionNumber,UserID,Age,NumDependents,Latitude,Longitude,UserTenure,IsFraud
count,10966.000000,10966.000000,10966.000000,10966.000000,10923.000000,10923.000000,10966.000000,10966.000000
mean,9117.500091,2483.885282,2751.845887,1.995714,-30.363255,141.254786,60.786157,0.364308
std,5293.669640,1429.402615,9730.988917,1.408035,6.962819,11.268395,34.254477,0.481257
min,1.000000,1.000000,-68.000000,0.000000,-41.640079,-112.026050,1.000000,0.000000
25%,4516.250000,1253.000000,26.000000,1.000000,-37.020100,142.702789,31.000000,0.000000
50%,9120.500000,2471.000000,34.000000,2.000000,-31.840233,144.964600,61.000000,0.000000
75%,13710.750000,3727.000000,43.000000,3.000000,-25.042261,145.612793,90.000000,1.000000
max,18277.000000,5000.000000,67000.000000,4.000000,57.851580,149.012375,119.000000,1.000000


In [5]:
test.head(50)

,TransactionNumber,UserID,Age,Gender,Occupation,EducationLevel,MaritalStatus,NumDependents,Income,Expenditure,...,TransactionAmount,MerchantID,TransactionType,TransactionLocation,DeviceType,Latitude,Longitude,EmailDomain,Terrorism,UserTenure
0,11854,963,35,man,Student,Bachelor,Single,4,53733.41 AUD,AU$ 29296.02,...,AU$ 225.64,M005,Payment,Darwin,Desktop,-37.020100,144.964600,chapmangabriel@outlook.com,True,39
1,2647,1693,23,Male,Professional,Master,Single,2,54856.77 AUD,34628.31 AUD,...,AU$ 658.1,M003,Purchase,Darwin,Desktop,-37.020100,144.964600,sjones@gmail.com,True,65
2,5945,4383,44,Male,Student,Bachelor,Single,2,59011.72 AUD,33312.46 AUD,...,AU$ 133.59,M004,Purchase,Adelaide,mob,-30.000233,136.209152,woodmaria@yahoo.com,False,95
3,6798,1350,40,Male,Student,High School,Married,3,AU$ 128795.4,67049.0 AUD,...,AU$ 6.74,M008,Withdrawal,Canberra,Mobile,-37.020100,144.964600,rthornton@gmail.com,False,85
4,12985,4145,18,Male,Professional,Bachelor,Married,3,AU$ 44506.03,22856.31 AUD,...,15.67 AUD,M008,Transfer,Perth,Tablet,-20.917574,142.702789,daniel61@outlook.com,True,102
5,10085,1981,34,Male,Professional,Bachelor,Married,0,48811.06 AUD,26419.17 AUD,...,149.79 AUD,M005,Withdrawal,Darwin,Desktop,-25.042261,117.793221,jack69@outlook.com,False,105
6,7794,1288,39,Female,Retired,Bachelor,Single,3,76495.07 AUD,33862.74 AUD,...,AU$ 380.94,M007,Purchase,Sydney,galaxys7,-37.020100,144.964600,quinnmatthew@outlook.com,True,63
7,16065,3492,28,Female,Student,Bachelor,Married,0,AU$ 80872.99,AED 138121.67,...,AED 142.6,M005,Purchase,Cbr,mob,-37.020100,144.964600,reginagreen@disposable.com,False,102
8,11557,1370,38,Female,Student,Bachelor,Single,3,86743.12 AUD,AU$ 39814.03,...,358.2 AUD,M002,Purchase,Canberra,Desktop,-20.917574,142.702789,johnhowe@securemail.com,False,92
9,5971,3702,23,Female,Student,High School,Married,3,117850.22 AUD,AU$ 66666.27,...,AU$ 544.78,M002,Purchase,Canberra,Mobile,-31.840233,145.612793,yfriedman@gmail.com,True,31
